# 概要

20211004 

本コードは画像AIのための池の画像のファイルリスト作成コードである<br>



# ライブラリのインポート

## Loggerのインポート

`MyLogger`は自作のロガークラスであり、`util/MyLogger.py`で実装されている。<br>

In [7]:
%matplotlib inline

from IPython.display import display

import argparse
import os
import sys
import re
import cv2
import shutil
import time as tm
import datetime as dt
import numpy as np
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
from PIL import Image


from utils import MyLogger

sns.set()

# Configファイルのロード

設定情報やハイパーパラメータをjsonからロードする。<br>
引数ではないが、利便性からargsに格納する。<br>

In [4]:
config_path = "configs/create_filelist"
config_filename = "east_3-2_final_settling_tank"

args = pd.read_json(Path(config_path, f"{config_filename}.json"), typ = 'series', encoding = "utf-8")
display(args)

seed                                                    20190517
indir          //10.101.65.115/広島西部/002_広島西部水資源再生センター/02_data...
outdir_log                                                  None
start_index                                   2019-12-1 00:00:00
end_index                                     2019-12-4 23:59:59
dtype: object

# アウトプットディレクトリの生成


In [5]:
# if args.outdir_log:
#     outdir_log = Path("logs", "crop_image", args.outdir_log)
# else :
#     outdir_log = Path("logs", "crop_image", config_filename)

outdir_log = Path("results", "create_filelist", config_filename)
outdir_log.mkdir(parents=True, exist_ok = True)
args.to_csv(Path(outdir_log, config_filename + ".csv"))
display('Log files are saved in {}'.format(outdir_log))

'Log files are saved in logs\\create_filelist\\east_3-2_final_settling_tank'

# Loggerの設定

MyLoggerクラスによって汎用ロガーを作成する。<br>
標準出力にINFO情報までを出力し、`root`と`filename`に指定したファイルにDEBUG情報まで出力する。<br>
moduleNmaeは`__name__`とすることでmainを指定できる。

In [8]:
path_log = Path('logs/')
path_log.mkdir(parents = True, exist_ok = True)

from logging import ERROR, INFO
ml = MyLogger(root = path_log,
              filename = "create_filelist.log",
              stream_level = INFO,
              moduleName = __name__)

logger = ml.getModuleLogger()

logger.info("start script")
logger.info(f"config file name :{config_filename}")

In [6]:
def get_filelist(file_path):
    file_list = deque(maxlen = 10**6)
#     for root, dirs, files in tqdm(os.walk(file_path)):
#         for file in files:
#             filetime_ep = os.path.getmtime(Path(root, file))
#             filetime = dt.datetime.fromtimestamp(filetime_ep)
#             filetime_st = filetime.strftime('%Y-%m-%d %H:%M:%S')
#             file_list.append([Path(root, file), filetime])
    
    for file in tqdm(Path(file_path).glob("**/*.jpg")):
        if file.is_file() == True:
            filetime_ep = os.path.getmtime(Path(file.parent, file.name))
            filetime = dt.datetime.fromtimestamp(filetime_ep)
            filetime_st = filetime.strftime('%Y-%m-%d %H:%M:%S')
            file_list.append([file, filetime])
    
    
    file_list_pd = pd.DataFrame(list(file_list), columns = ["filename", "datetime"])
    file_list_pd = file_list_pd.set_index("datetime")
    file_list_pd = file_list_pd.sort_index()
    
    return file_list_pd

In [9]:
t_s = tm.time()

file_list_path = Path(outdir_log, "file_list.csv")

file_list = get_filelist(args.indir)
file_list.to_csv(file_list_path)

duration = tm.time() - t_s
logger.info("get file list : elapsed time is {:.2f} [s]".format(duration))

display(file_list.shape)
display(file_list.index.dtype)
display(file_list.dtypes)
display(file_list.head())
display(file_list.tail())

209569it [06:44, 517.68it/s]


(209569, 1)

dtype('<M8[ns]')

filename    object
dtype: object

,filename
datetime,
2018-03-02 10:40:10,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:10:46,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:10:56,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:11:06,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2018-08-21 09:11:16,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


,filename
datetime,
2019-12-05 13:16:18,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:28,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:38,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:48,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...
2019-12-05 13:16:58,\\10.101.65.115\広島西部\002_広島西部水資源再生センター\02_data...


In [10]:
file_list.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 209569 entries, 2018-03-02 10:40:10 to 2019-12-05 13:16:58
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   filename  209569 non-null  object
dtypes: object(1)
memory usage: 3.2+ MB
